In [1]:
from Curation.CurationMain import *

In [2]:
import numpy as np
import math
import traceback
import urllib
from bs4 import BeautifulSoup
import html5lib
#메캅
import mecab_util
import mecab
mcb = mecab.MeCab()
#정규식
import re
#슬랙
from SendSlack import SlackClient
global sc
import Parallel
sc = SlackClient()
#패텍
import sys
import scipy
from gensim.models import FastText

In [3]:
algoname = 'Baseline'

In [4]:
#보조함수들

#doc-frq를 구해서 dic_df에 저장하는 알고리즘
#포스태그된 list와 doc-frq를 저장할 dict를 입력받는다.
#그리고 해당 행의 tf를 함께 구한다.
def calculate_dicdf(poses, dic_df):
    dic_tf = {}
    for p in poses:
        if p in dic_df:
            dic_df[p]['df'] += 1            
        else:
            dic_df[p] = {'key':p, 'df':1}
        if p in dic_tf:
            dic_tf[p] += 1
        else:
            dic_tf[p] = 1
    return dic_tf

#dic_df와 tf를 입력받아 tfidf 값을 리턴한다.
def calc_tfidf(dic_df, tf):
    tfidf = {}    
    for k, v in tf.items():        
        tfidf[k] = {'key':k, 'tf':v, 'tfidf':v/dic_df[k]['idf'], 'idf':dic_df[k]['idf']}
    return tfidf

#docvec을 fasttext모델과 postag ordered로 구한다.
def calcdocvec(model, ls_ordered):
    ls = ls_ordered[:10]    
    
    #날아간 영어기사에 대한 임이의 값
    if len(ls) == 0:
        return model['this is an english random word pos']
    
    max_tfidf_val = max( [l['tfidf'] for l in ls] )
    
    vec = model[ls[0]['key']]
    for tfidf in ls[:1]:
        vec = vec + (model[tfidf['key']] * (tfidf['tfidf']/max_tfidf_val) )
    return vec / len(ls)
    #return vec

In [5]:
#text를 입력받으면, 그녀석의 docvec을 구해준다.
#필요한것 : fasttext model, doc-frq-dict, 최대 df, preprocess를 위한 copyright제거기, 불필요한 postag제거기
def make_myvec(text, model, dic_df, maxdf, remove_copyright, stop_poses):
    dicpos = {}
    
    #저작권 제거
    tt = remove_copyright.sub('',text)
    
    #포스태깅
    lsp = mecab_util.PosWithSpace(tt)
    
    #stop_pos 제거
    lsp = [p for p in lsp if False not in [None == r.match(p[1]) for r in stop_poses]]
    
    #포스태그 합침
    lsp = [p[0]+'/'+p[1] for p in lsp]
    
    #트리밍
    lsp = [p for p in lsp if p != ' /SPACE']
        
    for p in lsp:        
        if p in dicpos:
            dicpos[p]['tf'] += 1
        else:
            dicpos[p] = {'key':p, 'tf':1}
    
    for k, v in dicpos.items():
        dd = dic_df.get(p, None)

        #자기의 df
        mydf = dd['df'] if dd != None else 1
        idf = mydf/maxdf
        v['idf'] = idf
        v['tfidf'] = v['tf']/v['idf']
        
    lspos = sorted([v for k, v in dicpos.items()], key=lambda x : x['tfidf'], reverse=True)    
    #print(lspos[:5])
    #print([l['tfidf'] for l in lspos[:5]] )
    max_tfidf_val = max( [l['tfidf'] for l in lspos] )
    #print([p['key'] for p in lspos[:10]])
    vecs = [(p, model[p['key']]) for p in lspos[:10]]    
    vec = vecs[0][1] * (vecs[0][0]['tfidf']/ max_tfidf_val)
    for v in vecs[1:]:
        vec = vec + v[1] * (v[0]['tfidf'] / max_tfidf_val)        
    return vec/len(vecs)

In [6]:
#similar_text를 구해서 올려준다.
#0 구할 문장
#1 'docvec' 컬룸을 가진 글 dataframe
#2 최대 몇개
def similar(text, df_feed, topn=10):
    vec = make_myvec(text, model, dic_df, maxdf, remove_copyright, stop_poses)
    
    df_feed['cos_sim'] = df_feed['docvec'].apply(lambda x : scipy.spatial.distance.cosine(x, vec))
    dfdf = df_feed.sort_values('cos_sim', ascending=False)
    return dfdf[:topn]

In [7]:
#일단 저작권을 어떻게든 떼보려는 시도
remove_copyright = re.compile('^\s*[▲]*\s*[\(\[](.*?)[\)\]]\s*')

#자를놈들, 명사 아닌놈, NNB, NR, NP
stop_poses = [re.compile('^[^N]'), re.compile('[^A-Z]*NNB[^A-Z]*'), re.compile('[^A-Z]*NR[^A-Z]*'),re.compile('[^A-Z]*NP[^A-Z]*')]

In [8]:
#만들어진 dataframe 을 serializable하게 만든다.
def make_df_serializable(df_in):
    df = df_in[:]
    for c in df.columns:
        ls = list(df[c].apply(type).unique())
        if list in ls or dict in ls or set in ls:
            df[c] = df[c].apply(lambda x : json.dumps(x, ensure_ascii=False, default=str) if x != None else x)
    return df

In [9]:
#실제 learning함수
def Learning():
    #피드 리퀘스트 아티클을 다운받은다
    df_origin = DocsFeedRequesters()
    df = df_origin[:]
    
    #피드 등장할 아티클을 다운받는다
    df_feed_origin = DocsFeedContents()
    df_feed = df_feed_origin[:]
    
    df = df[df['contents'].notna()]
    df_feed = df_feed[df_feed['contents'].notna()]
    
    #html tags와 저작권 표기 제거함
    df['preprocessed_contents'] = df['contents'].apply(lambda x : remove_copyright.sub('', BeautifulSoup(x,'html5lib').text))

    df_feed['preprocessed_contents'] = df_feed['contents'].apply(lambda x : remove_copyright.sub('', BeautifulSoup(x,'html5lib').text))
    
    #mecab_util 돌리고 단어/형태소명 으로 바꿈
    df['pos'] = df['preprocessed_contents'].apply(lambda x : [f"{t[0]}/{t[1]}" for t in mecab_util.PosWithSpace(x, mcb)])
    df_feed['pos'] = df_feed['preprocessed_contents'].apply(lambda x : [f"{t[0]}/{t[1]}" for t in mecab_util.PosWithSpace(x, mcb)])
    
    #스페이스바 제거
    df['pos_trim'] = df['pos'].apply(lambda x : [t for t in x if t != ' /SPACE'])
    df_feed['pos_trim'] = df_feed['pos'].apply(lambda x : [t for t in x if t != ' /SPACE'])
    
    #코퍼스 생성
    corpus_list = list(df['pos']) + list(df_feed['pos'])
    
    #w2v 제작
    import NLDate
    vectorsz = 20
    window = 10
    model = FastText(size=vectorsz, window=5, min_count=1)
    print(NLDate.Date(''))
    print("begin build vocab")
    model.build_vocab(sentences=corpus_list)
    print(NLDate.Date(''))
    print("begin train")
    model.train(sentences=corpus_list, total_examples=len(corpus_list), epochs=10)
    print(NLDate.Date(''))
    print("fin")
    
    print(NLDate.Date(''))
    print("begin preprocess")
    #자를 태그 찾아서 잘라냄
    df['filtered_pos'] = df['pos_trim'].apply(lambda x : [t for t in x if False not in [None == r.match(t.split('/')[1]) for r in stop_poses]])
    print(NLDate.Date(''))
    print("fin")

    #doc-frq를 구해서 저장한다.
    dic_df = {}
    print(NLDate.Date(''))
    print("begin doc-frq")
    df['tf'] = df['filtered_pos'].apply(lambda x : calculate_dicdf(x, dic_df))
    print(NLDate.Date(''))
    print("fin")
    
    #idf 계산
    maxdf = len(df)
    for k, v in dic_df.items():
        v['idf'] = v['df']/maxdf

    #1개의 dic_df 내용
    print(dic_df[list(dic_df.keys())[0]])
    
    #tfidf를 계산한다.
    print(NLDate.Date(''))
    print("begin tfidf")
    df['tfidf'] = df['tf'].apply(lambda x : calc_tfidf(dic_df, x))
    print(NLDate.Date(''))
    print("fin")
    
    #각 기사마다 tfidf 순으로 포스태깅단어를 소팅해서 알려준다.
    df['tfidf_sorted'] = df['tfidf'].apply(lambda x : sorted([v for k, v in x.items()], key=lambda tv : tv['tfidf'], reverse=True))
    
    print(df['tfidf_sorted'].iloc[0][0])
    
    print(NLDate.Date(''))
    print("begin docvec-all")
    df['docvec'] = df['tfidf_sorted'].apply(lambda x : calcdocvec(model, x) )

    print(NLDate.Date(''))
    print("fin")
    
    #feed의 docvec도 구한다.
    print(NLDate.Date(''))
    print("begin docvec-feed")
    df_feed['docvec'] = df_feed['contents'].apply(lambda x : make_myvec( BeautifulSoup(x, 'html5lib').text, model, dic_df, maxdf, remove_copyright, stop_poses))
    print(NLDate.Date(''))
    print("fin")
    
    df = df[df['docvec'].notna()]
    df_feed = df_feed[df_feed['docvec'].notna()]
    
    #모델 저장
    #0. 기존 모델 클리어
    ClearAlgorithmCache(algoname)
    
    #1. 만든 fasttext w2v 모델을 디스크에 저장하고 upload한다.
    print("save fasttext-model")
    model.save('/tmp/fasttext.model')
    SaveAlgorithmCache(algoname, 'cache/fasttext.model', '/tmp/fasttext.model', filetype='file')
    
    #2. dic_df 를 저장한다
    print("save dic_df")
    SaveAlgorithmCache(algoname, 'cache/dic_df.json', json.dumps(dic_df, ensure_ascii=False))
    
    #3. 데이터프레임 만들어놓은것들 저장한다
    print("save dataframes to parquet")    
    
    #여기서 pickling 해야함!!!!
    try:
        df['docvec'] = df['docvec'].apply(lambda x : json.dumps([float(t) for t in list(x)]) )
        df_feed['docvec'] = df_feed['docvec'].apply(lambda x : json.dumps([float(t) for t in list(x)]))
    except:
        print(traceback.format_exc())
    
    
    df = make_df_serializable(df)
    df_feed = make_df_serializable(df_feed)
    
    try:
        SaveAlgorithmCache(algoname, 'cache/df_feed.parquet', df_feed, filetype='dataframe')
        SaveAlgorithmCache(algoname, 'cache/df.parquet', df, filetype='dataframe')
    except:
        print(traceback.format_exc())
    
    print("save fin")
    
    return model, df, df_feed, dic_df
    

In [12]:
df_feed.to_parquet('df_feedtest.parquet')

In [11]:
df.to_parquet('dftest.parquet')

In [10]:
model, df, df_feed, dic_df = None, None, None, None
if __name__ == "__main__":
    model, df, df_feed, dic_df = Learning()

/opt/conda/lib/python3.7/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


2020-03-08 17:56:22.119490+09:00
begin build vocab
2020-03-08 17:57:56.032709+09:00
begin train
2020-03-08 18:36:21.447929+09:00
fin
2020-03-08 18:36:21.448223+09:00
begin preprocess
2020-03-08 18:39:06.044373+09:00
fin
2020-03-08 18:39:06.044615+09:00
begin doc-frq
2020-03-08 18:39:16.442438+09:00
fin
{'key': '배우/NNG', 'df': 6790, 'idf': 0.04953673305610272}
2020-03-08 18:39:16.490034+09:00
begin tfidf
2020-03-08 18:39:27.913385+09:00
fin
{'key': '차문현/NNP', 'tf': 1, 'tfidf': 137070.0, 'idf': 7.295542423579193e-06}
2020-03-08 18:39:38.216526+09:00
begin docvec-all


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:36: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:38: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:32: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


2020-03-08 18:39:41.197506+09:00
fin
2020-03-08 18:39:41.197806+09:00
begin docvec-feed


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:41: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


2020-03-08 18:44:59.026440+09:00
fin
save fasttext-model
save dic_df
save dataframes to parquet


Exception in thread Thread-61:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/tf/notebooks/libs/AWS/S3.py", line 335, in Upload_Dataframe
    s3.Object(bucketname, s3path).put(Body=data_bytes, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/boto3/resources/factory.py", line 520, in do_action
    response = action(self, *args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/boto3/resources/action.py", line 83, in __call__
    response = getattr(parent.meta.client, operation_name)(**params)
  File "/opt/conda/lib/python3.7/site-packages/botocore/client.py", line 276, in _api_call
    return self._make_api_call(operation_name, kwargs)
  File "/opt/conda/lib/python3.7/site-packages/botocore/client.py", line 586, in _make_api_call
    raise error_class(parsed_response, operat

save fin


Exception in thread Thread-62:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/tf/notebooks/libs/AWS/S3.py", line 335, in Upload_Dataframe
    s3.Object(bucketname, s3path).put(Body=data_bytes, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/boto3/resources/factory.py", line 520, in do_action
    response = action(self, *args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/boto3/resources/action.py", line 83, in __call__
    response = getattr(parent.meta.client, operation_name)(**params)
  File "/opt/conda/lib/python3.7/site-packages/botocore/client.py", line 276, in _api_call
    return self._make_api_call(operation_name, kwargs)
  File "/opt/conda/lib/python3.7/site-packages/botocore/client.py", line 586, in _make_api_call
    raise error_class(parsed_response, operat

In [ ]:
ftmodel = None
dic_df = None

def Init():
    path = LoadAlgorithmCache(algoname, 'w2v_fasttext.model', filetype='file')
    ftmodel = FastText.load(path)
    dic_df = json.loads(LoadAlgorithmCache(algoname, 'dic_df.json'))
    df = pd.DataFrame(LoadAlgorithmCache(algoname, 'dic_df.json'))


#def Curation(**kwargs):
    
    